# Multi-Agent Systems with Semantic Kernel

Welcome to this workshop on multi-agent systems using Semantic Kernel! In this notebook, we'll explore how to create, configure, and orchestrate multiple agents to collaborate in solving complex tasks.

## What You'll Learn
- How to transition from single agents to multi-agent systems
- The fundamentals of AgentGroupChat for agent collaboration
- How to design specialized agents with different roles
- Strategies for managing agent conversations and turn-taking
- Techniques for controlling conversation flow and termination

Let's start by setting up our environment.

In [1]:
import os
import asyncio
from dotenv import load_dotenv

# Import Semantic Kernel components
import semantic_kernel as sk
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import KernelArguments
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
    SequentialSelectionStrategy,
    DefaultTerminationStrategy,
)

from semantic_kernel.agents.strategies.selection.selection_strategy import (
    SelectionStrategy,
)


# Load environment variables from .env file
load_dotenv()

print("Environment set up successfully!")

Environment set up successfully!


Let's create our helper function for setting up a kernel with AI services, similar to what we used in the previous notebook:

In [2]:
def create_kernel_with_service(service_id):
    kernel = sk.Kernel()
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            deployment_name=os.getenv("AZURE_OPENAI_MODEL_DEPLOYMENT_NAME"),
            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
            endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
        )
    )
    return kernel

# Create our kernel
kernel = create_kernel_with_service(service_id="multi-agent")
print("Kernel created successfully!")

Kernel created successfully!


In [3]:
# Create our kernel
kernel = create_kernel_with_service(service_id="multi-agent")


# Create the agents with the initialized kernel
agent_factual = ChatCompletionAgent(
    kernel=kernel,
    name="Factual",
    instructions="You are a fact-based agent who provides accurate and concise information. Always stick to verified facts and cite sources when possible. Keep your responses very concise, clear and straightforward.",
)

agent_creative = ChatCompletionAgent(
    kernel=kernel,
    name="Creative",
    instructions="You are a creative agent who thinks outside the box. Offer innovative perspectives and unique ideas. Feel free to brainstorm and suggest creative solutions. Keep your responses very concise, imaginative and engaging.",
)

# Create a simple group chat with these agents
simple_group_chat = AgentGroupChat(
    agents=[agent_factual, agent_creative],
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=4),
)

print(f"Created a simple group chat with {len(simple_group_chat.agents)} agents")
print(f"Maximum iterations: {simple_group_chat.termination_strategy.maximum_iterations}")

Created a simple group chat with 2 agents
Maximum iterations: 4


## 1. Introduction to Multi-Agent Systems

### From Single Agents to Agent Systems

While single agents are powerful for many applications, some tasks benefit from having multiple specialized agents working together. Multi-agent systems provide several advantages:

1. **Specialization**: Each agent can focus on what it does best
2. **Role separation**: Different agents can take on different roles (critic, creator, researcher, etc.)
3. **System reliability**: If one agent struggles, others can compensate
4. **Complex problem-solving**: Breaking down problems into sub-tasks handled by different agents
5. **Natural collaboration flow**: Mimicking how humans collaborate in teams

Think of a multi-agent system like a team meeting where each participant has different expertise and contributes to the discussion based on their strengths.

### AgentGroupChat Fundamentals

The `AgentGroupChat` class is the core component for multi-agent collaboration in Semantic Kernel. It provides a framework for multiple agents to interact in a structured conversation.

Key components of an `AgentGroupChat` include:

1. **Agents**: The collection of agents participating in the conversation
2. **Selection Strategy**: Determines which agent speaks next
3. **Termination Strategy**: Decides when the conversation should end
4. **Chat History**: The shared conversation context visible to all agents


The conversation flow is typically:

1. User adds a message to the group chat
2. Selection strategy chooses which agent responds next
3. Selected agent processes the conversation history and generates a response
4. Response is added to the chat history
5. Termination strategy checks if conversation should end
6. If not terminated, process repeats from step 2

Let's start with a simple example: creating two basic agents and connecting them through an AgentGroupChat.

In [4]:
# Create two simple agents with different perspectives
agent_factual = ChatCompletionAgent(
    kernel=kernel,
    name="Factual",
    instructions="You are a fact-based agent who provides accurate and concise information. Always stick to verified facts and cite sources when possible. Keep your responses very concise, clear and straightforward.",
)

agent_creative = ChatCompletionAgent(
    kernel=kernel,
    name="Creative",
    instructions="You are a creative agent who thinks outside the box. Offer innovative perspectives and unique ideas. Feel free to brainstorm and suggest creative solutions. Keep your responses very concise, imaginative and engaging.",
)

# Create a simple group chat with these agents
simple_group_chat = AgentGroupChat(
    agents=[agent_factual, agent_creative],
    # Use sequential selection (round-robin) and limit to 4 total responses
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=4),
)

print(f"Created a simple group chat with {len(simple_group_chat.agents)} agents")
print(f"Maximum iterations: {simple_group_chat.termination_strategy.maximum_iterations}")

Created a simple group chat with 2 agents
Maximum iterations: 4


In [5]:
async def run_group_chat(chat, user_message):
    """Run a multi-agent conversation and display the results.

    Args:
        chat: The AgentGroupChat instance
        user_message: The initial user message to start the conversation

    Returns:
        The chat history containing all messages
    """
    # Create a new chat history if needed
    if not hasattr(chat, "history") or chat.history is None:
        # Some versions of AgentGroupChat might not initialize history
        chat_history = ChatHistory()
        chat.history = chat_history

    # Add the user message to the chat
    await chat.add_chat_message(message=user_message)
    print(f"\nUser: {user_message}\n")
    print("=== Beginning Agent Collaboration ===")

    # Track which agent is speaking for formatting
    current_agent = None

    # Invoke the chat and process agent responses
    try:
        async for response in chat.invoke():
            if response is not None and response.name:
                # Add a separator between different agents
                if current_agent != response.name:
                    current_agent = response.name
                    print(f"\n## {response.name}:\n{response.content}")
                else:
                    # Same agent continuing
                    print(f"{response.content}")

        print("\n=== Agent Collaboration Complete ===")
    except Exception as e:
        print(f"Error during chat invocation: {str(e)}")

    # Reset is_complete to allow for further conversations
    chat.is_complete = False

    return chat.history

In [6]:
# Test the simple group chat with a question
question = "What are some approaches to address climate change?"
chat_history = await run_group_chat(simple_group_chat, question)


User: What are some approaches to address climate change?

=== Beginning Agent Collaboration ===

## Factual:
There are several approaches to address climate change, including:

1. **Reducing Greenhouse Gas Emissions**:
   - Transition to renewable energy (solar, wind, hydro).
   - Improve energy efficiency in buildings, transportation, and industries.
   - Promote sustainable agriculture and forestry practices.
   - Implement carbon pricing (carbon taxes or cap-and-trade systems).

2. **Carbon Capture and Storage (CCS)**:
   - Develop technologies to capture and store CO2 from industrial processes and power plants.

3. **Adopting Climate Resilient Practices**:
   - Enhance infrastructure to withstand extreme weather.
   - Support adaptation strategies for vulnerable communities.

4. **Investing in Research and Innovation**:
   - Develop new technologies for clean energy and sustainable practices.

5. **International Cooperation**:
   - Engage in global agreements like the Paris Accor

In a multi-agent system, each agent should have a clearly defined role and purpose. Here are some common agent roles you might implement:

| Role Type | Purpose | Example Instructions |
|-----------|---------|---------------------|
| **Researcher** | Provides factual information | Focus on accurate information with sources |
| **Creative** | Generates novel ideas | Think outside conventional boundaries |
| **Critic** | Analyzes and critiques | Identify weaknesses and suggest improvements |
| **Summarizer** | Condenses information | Extract key points concisely |
| **Moderator** | Guides conversation | Keep discussion on track and productive |
| **Domain Expert** | Provides specialized knowledge | Share deep expertise in a specific field |

Let's create a more sophisticated team of agents with specialized roles:

In [7]:
# Create a team of specialized agents

# 1. Researcher who provides factual information
researcher = ChatCompletionAgent(
    kernel=kernel,
    name="Researcher",
    instructions="""You are a thorough researcher who provides factual information and analysis.
    
    Your responsibilities:
    - Provide accurate, factual information on the topic
    - Analyze questions from multiple angles
    - Consider historical context and current understanding
    - Be objective and balanced in your assessment
    - Acknowledge limitations in current knowledge when appropriate
    
    Always strive for accuracy over speculation. Keep your responses very concise, clear and straightforward.""",
)

# 2. Creative thinker who generates innovative ideas
innovator = ChatCompletionAgent(
    kernel=kernel,
    name="Innovator",
    instructions="""You are an innovative thinker who generates novel ideas and perspectives.
    
    Your responsibilities:
    - Suggest unique approaches and solutions
    - Think beyond conventional boundaries
    - Make unexpected connections between concepts
    - Offer imaginative scenarios and possibilities
    - Propose 'what if' scenarios to expand thinking
    
    Don't be constrained by traditional thinking - be bold and creative. Keep your responses very concise, imaginative and engaging.""",
)

# 3. Critic who evaluates ideas critically
critic = ChatCompletionAgent(
    kernel=kernel,
    name="Critic",
    instructions="""You are a thoughtful critic who evaluates ideas and identifies potential issues.
    
    Your responsibilities:
    - Analyze the strengths and weaknesses of proposals
    - Identify potential problems or limitations
    - Challenge assumptions constructively
    - Suggest improvements to ideas
    - Consider practical implementation challenges
    
    Be constructive in your criticism - your goal is to improve ideas, not dismiss them. Keep your responses very concise, clear and straightforward.""",
)

# 4. Synthesizer who brings ideas together
synthesizer = ChatCompletionAgent(
    kernel=kernel,
    name="Synthesizer",
    instructions="""You are a skilled synthesizer who integrates diverse perspectives into coherent conclusions.
    
    Your responsibilities:
    - Identify common themes across different viewpoints
    - Reconcile apparently conflicting ideas when possible
    - Create a balanced, integrated perspective
    - Summarize key points from the discussion
    - Draw reasonable conclusions from the collective input
    
    Your goal is to bring together different perspectives into a coherent whole. Keep your responses very concise, clear and straightforward.""",
)

# Create a group chat with all specialized agents
expert_team = AgentGroupChat(
    agents=[researcher, innovator, critic, synthesizer],
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=DefaultTerminationStrategy(
        maximum_iterations=8
    ),  # 2 rounds of all 4 agents
)

print(f"Created an expert team with {len(expert_team.agents)} specialized agents")

Created an expert team with 4 specialized agents


Now let's test our expert team with the same climate change question to see how their specialized roles affect the collaboration:

In [8]:
chat_history = await run_group_chat(expert_team, question)


User: What are some approaches to address climate change?

=== Beginning Agent Collaboration ===

## Researcher:
Approaches to address climate change can be divided into mitigation, adaptation, and governance/policy initiatives. Key strategies include:

### **Mitigation Approaches**
- **Reducing Greenhouse Gas Emissions:** Transitioning to renewable energy sources (solar, wind, and hydropower), improving energy efficiency, and electrifying transportation.
- **Carbon Capture and Storage (CCS):** Capturing CO2 emissions from industrial processes and storing them in geological formations.
- **Reforestation and Afforestation:** Planting trees to absorb CO2 and restore natural ecosystems.
- **Sustainable Agriculture:** Reducing methane emissions from livestock, improving soil health, and adopting regenerative farming practices.

### **Adaptation Approaches**
- **Climate-Resilient Infrastructure:** Strengthening infrastructure to withstand extreme weather events, rising sea levels, and chan

Compare the two approaches:

1. **Simple Group Chat (2 agents)**:
   - Basic factual and creative perspectives
   - Limited interaction depth
   - Simple back-and-forth exchange

2. **Expert Team (4 specialized agents)**:
   - Multiple specialized perspectives
   - More comprehensive analysis
   - Natural progression from facts → ideas → critique → synthesis

This demonstrates how designing agents with complementary roles can create more valuable interactions and outputs.

## 2. Introduction Agent Strategies

In the previous section, we created basic multi-agent systems using `AgentGroupChat` with default strategies. Now let's explore how to build more sophisticated agent collaborations by configuring agents with specialized roles and customizing their interactions.

### Initializing Multiple Specialized Agents

When building effective multi-agent systems, the way you design and initialize your agents is critical. Each agent should have:

1. **Clear purpose**: A well-defined role and responsibility
2. **Specialized instructions**: Guidance on how to approach problems from its unique perspective
3. **Complementary skills**: Abilities that work well with other agents' capabilities
4. **Communication style**: How it should present information to other agents

Let's create a team of specialized agents for a specific task: collaborative writing and editing.

In [9]:
# Create a writer agent that generates content
writer = ChatCompletionAgent(
    kernel=kernel,
    name="Writer",
    instructions="""You are a creative writer who crafts engaging content.
    
    Your role in this collaboration:
    1. Generate original content based on the topic provided
    2. Apply creative storytelling techniques to engage readers
    3. Incorporate feedback from the editor and fact-checker to improve your writing
    4. Revise content to address issues raised by other team members
    5. Focus on creating a compelling narrative voice and structure
    
    When responding to feedback:
    - Be open to constructive criticism
    - Explain your creative choices when relevant
    - Incorporate suggestions that improve the content
    
    Always strive to maintain the core message while making the content more engaging and effective. Keep your responses very concise, imaginative and engaging.""",
)

# Create an editor agent that improves the writing
editor = ChatCompletionAgent(
    kernel=kernel,
    name="Editor",
    instructions="""You are a meticulous editor who improves content quality and clarity.
    
    Your role in this collaboration:
    1. Review content for clarity, coherence, and flow
    2. Identify and fix grammatical, structural, or stylistic issues
    3. Suggest improvements to enhance readability and impact
    4. Ensure the content meets its intended purpose and audience needs
    5. Maintain consistent voice and tone throughout
    
    When providing feedback:
    - Be specific about what needs improvement and why
    - Offer constructive suggestions rather than just criticism
    - Consider both micro (sentence-level) and macro (structure) improvements
    - Balance preserving the writer's voice with improving the content
    
    Your goal is to elevate the writing while respecting the writer's intent and style. Keep your responses very concise, clear and straightforward.""",
)

# Create a fact-checker agent that ensures accuracy
fact_checker = ChatCompletionAgent(
    kernel=kernel,
    name="FactChecker",
    instructions="""You are a thorough fact-checker who ensures accuracy and credibility.
    
    Your role in this collaboration:
    1. Verify factual claims in the content
    2. Identify potential inaccuracies or misleading statements
    3. Suggest corrections for any factual errors
    4. Recommend additional context where needed for accuracy
    5. Ensure the content is truthful and well-supported
    
    When providing feedback:
    - Focus on accuracy rather than style or structure
    - Explain why a statement might be problematic
    - Provide correct information to replace inaccuracies
    - Consider potential sources of factual support
    
    Your goal is to ensure the content maintains high standards of accuracy and integrity. Keep your responses very concise, clear and straightforward.""",
)

print("Created specialized writing team agents successfully!")

Created specialized writing team agents successfully!


### Setting Up a Basic Group Chat

Now that we have our specialized agents, let's set up an `AgentGroupChat` to orchestrate their collaboration. We need to consider:

1. **Agent order**: The sequence in which agents participate
2. **Maximum iterations**: How many turns the conversation should take
3. **Turn-taking logic**: How to determine which agent speaks next

For this example, we'll use a simple sequential approach where agents take turns in a fixed order.

In [10]:
# Create a writing team chat with sequential selection strategy
writing_team = AgentGroupChat(
    agents=[writer, editor, fact_checker],
    # Specify that the writer should start
    selection_strategy=SequentialSelectionStrategy(initial_agent=writer),
    # Limit to 9 turns total (3 rounds × 3 agents)
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=9),
)

print("Writing team chat created with sequential selection strategy")
print(f"Initial agent: {writing_team.selection_strategy.initial_agent.name}")
print(f"Maximum iterations: {writing_team.termination_strategy.maximum_iterations}")

Writing team chat created with sequential selection strategy
Initial agent: Writer
Maximum iterations: 9


The `SequentialSelectionStrategy` we're using has these properties:

- It follows a round-robin pattern, moving through agents in the order they were added
- You can specify an `initial_agent` to start the conversation
- It's predictable and ensures each agent gets a turn to contribute

Now let's test our writing team collaboration with a writing task:

In [11]:
# Test the writing team with a specific writing task
writing_task = "Write a short blog post about the impact of artificial intelligence on healthcare, focusing on recent advancements and ethical considerations."

# Run the group chat
chat_history = await run_group_chat(writing_team, writing_task)


User: Write a short blog post about the impact of artificial intelligence on healthcare, focusing on recent advancements and ethical considerations.

=== Beginning Agent Collaboration ===

## Writer:
**AI in Healthcare: Revolutionizing Medicine with a Human Touch**

Artificial intelligence is reshaping healthcare, delivering breakthroughs at a pace once thought impossible. From detecting diseases through advanced imaging to personalizing treatment plans via predictive analytics, AI is transforming patient outcomes. For instance, algorithms today can identify cancer with accuracy rivaling seasoned radiologists, or monitor chronic illnesses with wearable devices that provide real-time updates.

Yet, with innovation comes ethical dilemmas. Who is accountable when an algorithm errs? How do we prevent bias in AI systems that could deepen healthcare disparities? Balancing innovation with responsibility is the challenge—and opportunity—of our time.

As AI grows smarter, its success will depe

Did you notice the pattern in the conversation?

1. The **Writer** created the initial draft
2. The **Editor** reviewed it for clarity and style
3. The **FactChecker** verified the factual claims
4. The **Writer** revised based on feedback
5. The **Editor** reviewed the revised version
6. And so on...

This sequential approach mimics a real editorial process where content moves through different stages of review and revision.

### Simple Agent Collaboration Patterns

The sequential selection we just used is one of several agent collaboration patterns. Here are some common patterns you might use:

1. **Sequential (Round-Robin)**: Agents take turns in a fixed order
   - Pros: Simple, predictable, ensures each agent participates
   - Cons: Not responsive to content; might include irrelevant contributions

2. **Fixed Workflow**: Agents follow a specific sequence designed for a task
   - Pros: Mirrors real workflows (e.g., write → edit → approve)
   - Cons: Less flexible for unexpected situations

3. **Dynamic Selection**: An LLM decides which agent should respond next
   - Pros: More responsive to conversation needs; adaptable
   - Cons: Less predictable; requires careful prompt engineering

Let's implement a fixed workflow pattern for our writing team, where we explicitly define the sequence of agents based on the writing process:

### Exercise: Create Your First Multi-Agent Conversation

Now it's your turn to design a multi-agent system! 